In [17]:
import os
import json
import time
import re
import replicate
from jigsawstack import JigsawStack
import sys
sys.path.append('../')  # subir 1 nivel desde notebooks a src
from config.paths import SRC_DIR

# Comentario: variables de entorno esperadas
# - JIGSAWSTACK_API_TOKEN
# - REPLICATE_API_TOKEN
# - DEEPSEEK_EXTRACT_MODEL (slug exacto con :version)
# - DEEPSEEK_SYNTH_MODEL   (slug exacto con :version)
#   Ej: deepseek-ai/deepseek-v3.1 o deepseek-ai/deepseek-r1

In [18]:
JIGSAWSTACK_API_KEY = os.getenv("JIGSAWSTACK_API_TOKEN")
assert JIGSAWSTACK_API_KEY, "Falta JIGSAWSTACK_API_TOKEN en variables de entorno"

REPLICATE_API_TOKEN = os.getenv("REPLICATE_API_TOKEN")
assert REPLICATE_API_TOKEN, "Falta REPLICATE_API_TOKEN en variables de entorno"

DEEPSEEK_EXTRACT_MODEL = os.getenv("DEEPSEEK_EXTRACT_MODEL", "deepseek-ai/deepseek-v3.1")
DEEPSEEK_SYNTH_MODEL = os.getenv("DEEPSEEK_SYNTH_MODEL", "deepseek-ai/deepseek-v3.1")

client = replicate.Client(api_token=REPLICATE_API_TOKEN)
print("Extractor:", DEEPSEEK_EXTRACT_MODEL)
print("Synthesizer:", DEEPSEEK_SYNTH_MODEL)

Extractor: deepseek-ai/deepseek-v3.1
Synthesizer: deepseek-ai/deepseek-v3.1


In [19]:
jigsaw = JigsawStack(api_key=JIGSAWSTACK_API_KEY )

In [20]:
# --- Inputs (luego vendrán del CSV) ---
MOTO = {
    "brand": "Hero",
    "model": "Hunk 125R",
    "year": None,
    "country": "Colombia",
    "type": "urbana",  # urbana | deportiva | adventure | etc.
}

# --- Búsqueda diversificada (multi-moto / multi-país, sin hardcode de sitios) ---
# Comentario: ajusta estos valores según costo/latencia
PER_QUERY = 6
MAX_SOURCES = 24
SAFE_SEARCH = "moderate"

# Opcional (NO obligatorio): si quieres tunear el recall, puedes setear listas por país/cliente
INCLUDE_SITES: list[str] = []
EXCLUDE_SITES: list[str] = []

# Comentario: palabras clave por tipo (solo guía, no es hardcode de sitios)
TYPE_HINTS = {
    "urbana": ["opiniones", "dueños", "consumo real", "problemas", "mantenimiento", "repuestos"],
    "deportiva": ["sensaciones", "calor", "postura", "frenos", "consumo real", "fiabilidad"],
    "adventure": ["viajes", "carga", "off-road", "consumo", "falla", "mantenimiento"],
}

hints = TYPE_HINTS.get((MOTO.get("type") or "").lower(), ["opiniones", "dueños", "problemas", "consumo real"])

base = f"{MOTO['brand']} {MOTO['model']} {MOTO['country']}".strip()

# Comentario: construye query con exclusiones/inclusiones opcionales
site_filters = " ".join([f"site:{s}" for s in INCLUDE_SITES])
site_excludes = " ".join([f"-site:{s}" for s in EXCLUDE_SITES])

query_templates = [
    # experiencia de usuarios / problemas
    f"{base} {hints[0]} {hints[1]} {site_filters} {site_excludes}",
    # consumo real / uso diario
    f"{base} {hints[2]} {hints[3]} {site_filters} {site_excludes}",
    # comparaciones
    f"{base} comparativa vs {site_filters} {site_excludes}",
    # reventa / mercado
    f"{base} reventa precio usado {site_filters} {site_excludes}",
]

queries = [" ".join(q.split()) for q in query_templates]  # normaliza espacios

all_results = []
for q in queries:
    r = jigsaw.web.search({
        "query": q,
        "ai_overview": False,
        "safe_search": SAFE_SEARCH,
        "spell_check": True,
        "max_results": PER_QUERY,
        "auto_scrape": False,
        # "country_code": "...",  # opcional si lo quieres mapear por país
    })
    all_results.extend(r.get("results") or [])

# Deduplicar por URL
seen = set()
dedup = []
for item in all_results:
    url = item.get("url")
    if not url or url in seen:
        continue
    seen.add(url)
    dedup.append(item)

results = dedup[:MAX_SOURCES]

response = {
    "success": True,
    "query": " | ".join(queries),
    "results": results,
}

print("Moto:", MOTO)
print("Queries:\n- " + "\n- ".join(queries))
print("Total resultados (antes dedup):", len(all_results))
print("Total resultados (dedup):", len(dedup))
print("Usando fuentes:", len(results))
print("Primeras 10 URLs:")
for i, item in enumerate(results[:10]):
    print(f"{i}. {item.get('url')}")

Moto: {'brand': 'Hero', 'model': 'Hunk 125R', 'year': None, 'country': 'Colombia', 'type': 'urbana'}
Queries:
- Hero Hunk 125R Colombia opiniones dueños
- Hero Hunk 125R Colombia consumo real problemas
- Hero Hunk 125R Colombia comparativa vs
- Hero Hunk 125R Colombia reventa precio usado
Total resultados (antes dedup): 16
Total resultados (dedup): 10
Usando fuentes: 10
Primeras 10 URLs:
0. https://www.reddit.com/r/Colombia/comments/1kptshz/cr4_150_o_hero_hunk_125r_de_hero/
1. https://www.reddit.com/r/Colombia/comments/1kf0ahs/quienes_tengan_la_hero_hunk_125r_qu%C3%A9_tal_ha_salido/?tl=en
2. https://publimotos.com/actualidad/prueba-de-la-nueva-hero-125r-solo-un-fallo-para-ser-perfecta/
3. https://heromotos.com.co/wp-content/uploads/2025/03/Manual_Hunk125R_CO.pdf
4. https://www.galgo.com/co/motos/CO2961-hero-hunk-125-r
5. https://heromotos.com.co/urbanas/hunk-125r/
6. https://www.galgo.com/co/comparador/motos/CO2242-hero-eco-t-vs-CO2961-hero-hunk-125-r
7. https://demotos.com.co/noticias

In [ ]:
print()

In [21]:
response

{'success': True,
 'query': 'Hero Hunk 125R Colombia opiniones dueños | Hero Hunk 125R Colombia consumo real problemas | Hero Hunk 125R Colombia comparativa vs | Hero Hunk 125R Colombia reventa precio usado',
 'results': [{'title': 'r/Colombia on Reddit: Cr4 150 o hero hunk 125r de hero?',
   'url': 'https://www.reddit.com/r/Colombia/comments/1kptshz/cr4_150_o_hero_hunk_125r_de_hero/',
   'description': '**Question:**\nQue moto se comprarían entre estas dos y por qué? Quiero comprarme mi primera moto y la que me llama mas la atención es la hero hunk 125r, porque tiene un precio dentro de mi presupuesto, no me interesa andar rápido, porque es para desplazarme en la ciudad al trabajo, universidad y casa. Pero quiero que tenga un diseño bonito. Por otro lado no se que tan grabe sea para subir pendientes un motor 125… en caso de que lleve pato o que se yo. Denme su mas sincera opinión, gracias 😁\n\n**Answer:**\nlo malo de akt es que tienes a robarlas y están medio mal ensambladas por lo qu

In [22]:
# --- Preparar evidencia compacta (para evitar sesgo por una sola fuente) ---

def _clip(text: str | None, max_chars: int = 1600) -> str | None:
    # Comentario: recorta texto para mantener diversidad y bajar costo/tokens
    if not text:
        return None
    text = str(text).strip()
    return text[:max_chars]

def _domain(url: str | None) -> str | None:
    # Comentario: extrae dominio simple para clasificar fuentes
    if not url:
        return None
    m = re.search(r"https?://([^/]+)/", url)
    return m.group(1).lower() if m else None

compact = {
    "query": response.get("query"),
    "results": [],
}

results = response.get("results") or []
for i, r in enumerate(results):
    url = r.get("url")
    compact["results"].append(
        {
            "source_id": i,
            "title": r.get("title"),
            "url": url,
            "domain": _domain(url),
            "site_name": r.get("site_name"),
            "language": r.get("language"),
            "description": _clip(r.get("description"), 1200),
            # Comentario: con auto_scrape=False puede venir vacío; igual dejamos excerpt si existe
            "content_excerpt": _clip(r.get("content"), 1600),
        }
    )

print("Fuentes en results:", len(compact["results"]))
print("Primeras 10 URLs:")
for r in compact["results"][0:10]:
    print("-", r["source_id"], r.get("domain"), r["url"])

Fuentes en results: 10
Primeras 10 URLs:
- 0 www.reddit.com https://www.reddit.com/r/Colombia/comments/1kptshz/cr4_150_o_hero_hunk_125r_de_hero/
- 1 www.reddit.com https://www.reddit.com/r/Colombia/comments/1kf0ahs/quienes_tengan_la_hero_hunk_125r_qu%C3%A9_tal_ha_salido/?tl=en
- 2 publimotos.com https://publimotos.com/actualidad/prueba-de-la-nueva-hero-125r-solo-un-fallo-para-ser-perfecta/
- 3 heromotos.com.co https://heromotos.com.co/wp-content/uploads/2025/03/Manual_Hunk125R_CO.pdf
- 4 www.galgo.com https://www.galgo.com/co/motos/CO2961-hero-hunk-125-r
- 5 heromotos.com.co https://heromotos.com.co/urbanas/hunk-125r/
- 6 www.galgo.com https://www.galgo.com/co/comparador/motos/CO2242-hero-eco-t-vs-CO2961-hero-hunk-125-r
- 7 demotos.com.co https://demotos.com.co/noticias-de-motos/lanzamientos/hero-hunk-125r-economica-atrevida-y-balanceada/
- 8 www.youtube.com https://www.youtube.com/watch?v=-ksMg8cjits
- 9 publimotos.com https://publimotos.com/actualidad/tvs-raider-125-vs-hero-hunk-125r

In [24]:
# --- Pase 1: Extraer hechos/señales estructuradas (JSON) ---
# Comentario: leemos tu prompt base desde archivo para mantener el formato
prompt_path = os.path.join(SRC_DIR.parent, ".cursor", "reserach_used_prompt.md")
with open(prompt_path, "r", encoding="utf-8") as f:
    base_prompt = f.read()

# Comentario: schema mínimo (forzamos evidencia por item)
EXTRACTOR_INSTRUCTIONS = f"""
Eres un extractor de evidencia para investigación de motos.

CONTEXTO DE INPUT:
- brand: {MOTO.get('brand')}
- model: {MOTO.get('model')}
- year: {MOTO.get('year')}
- country: {MOTO.get('country')}
- type: {MOTO.get('type')}

REGLAS:
- Responde SOLO con JSON válido (sin markdown, sin texto extra).
- Usa ÚNICAMENTE EVIDENCIA_JSON.
- Para cada afirmación, incluye sources (lista de source_id) y confidence (Alta/Media/Baja).
- No mezcles países: si una fuente no es del país objetivo ({MOTO.get('country')}), marca country_scope="no_country".

Devuelve JSON con esta forma (puedes dejar null o [] si no hay evidencia):
{{
  "motorcycle": {{"brand": "{MOTO.get('brand')}", "model": "{MOTO.get('model')}", "year": {json.dumps(MOTO.get('year'))}, "country": "{MOTO.get('country')}"}},
  "facts": [
    {{"topic": "engine", "value": "...", "sources": [0,1], "confidence": "Media", "country_scope": "in_country"}}
  ],
  "sentiment": {{
    "overall": {{"summary": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}},
    "positives": [{{"value": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}}],
    "negatives": [{{"value": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}}]
  }},
  "recurrent_problems": [{{"value": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}}],
  "real_world_metrics": [{{"metric": "fuel_consumption", "value": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}}],
  "comparisons": [{{"vs": "...", "summary": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}}],
  "resale_signals": [{{"value": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}}],
  "mods": [{{"value": "...", "sources": [0], "confidence": "...", "country_scope": "in_country"}}],
  "missing_topics": ["..."]
}}
"""

extract_prompt = f"""
{EXTRACTOR_INSTRUCTIONS}

EVIDENCIA_JSON:
{json.dumps(compact, ensure_ascii=False)}
"""

print("Ejecutando extractor en Replicate:", DEEPSEEK_EXTRACT_MODEL)
t0 = time.time()
extract_out = client.run(
    DEEPSEEK_EXTRACT_MODEL,
    input={
        "prompt": extract_prompt,
        "temperature": 0.1,
        "max_new_tokens": 2000,
    },
)
print(f"Extractor listo en {time.time() - t0:.1f}s")

extract_text = "".join(extract_out) if isinstance(extract_out, list) else str(extract_out)

# Comentario: parse robusto (por si el modelo mete algo antes/después)
match = re.search(r"\{[\s\S]*\}", extract_text)
if not match:
    raise RuntimeError("No se encontró JSON en la salida del extractor")

signals = json.loads(match.group(0))
print("OK JSON. keys:", list(signals.keys()))

Ejecutando extractor en Replicate: deepseek-ai/deepseek-v3.1
Extractor listo en 38.4s
OK JSON. keys: ['motorcycle', 'facts', 'sentiment', 'recurrent_problems', 'real_world_metrics', 'comparisons', 'resale_signals', 'mods', 'missing_topics']


In [25]:
# --- Pase 2: Razonamiento + informe final (sin buscar) ---

SYNTH_INSTRUCTIONS = f"""
Eres un analista senior del mercado de motocicletas en {MOTO.get('country')}.

REGLAS:
- No busques en internet.
- No agregues datos externos.
- Usa SOLO el JSON de SEÑALES_JSON (derivado de evidencia).
- Adapta el análisis al país objetivo ({MOTO.get('country')}) y evita generalizar fuentes de otros países.

FORMATO OBLIGATORIO:
- Debes separar explícitamente:
  - Hechos (con evidencia): frases que estén soportadas por sources.
  - Inferencias (derivadas): conclusiones que conectan hechos; deben indicar en qué hechos se basan.

TRAZABILIDAD:
- Cada Hecho debe terminar con: "Evidencia: Source #X, #Y".
- Cada Inferencia debe terminar con: "Basado en: Source #X, #Y" y "Confianza: Alta/Media/Baja".

Si falta info, dilo en 'Limitaciones' y en missing_topics.
"""

synth_prompt = f"""
{base_prompt}

{SYNTH_INSTRUCTIONS}

SEÑALES_JSON:
{json.dumps(signals, ensure_ascii=False)}

NOTA:
- Prioriza el país objetivo ({MOTO.get('country')}). Si algo viene marcado como no_country, menciónalo y no lo generalices.
"""

print("Ejecutando síntesis en Replicate:", DEEPSEEK_SYNTH_MODEL)
t0 = time.time()
out = client.run(
    DEEPSEEK_SYNTH_MODEL,
    input={
        "prompt": synth_prompt,
        "temperature": 0.2,
        "max_new_tokens": 2600,
    },
)
print(f"Síntesis lista en {time.time() - t0:.1f}s")
print("".join(out) if isinstance(out, list) else out)

Ejecutando síntesis en Replicate: deepseek-ai/deepseek-v3.1
Síntesis lista en 55.0s
**Deep Sentiment Research: Hero Hunk 125R en Colombia**

**0. Segmento y tipo de moto**
- Tipo según BD: Urbana/Commuter
- Percepción de usuarios: Coincide con el uso reportado para desplazamientos urbanos (trabajo, universidad, casa). Su diseño "musculoso" y "agresivo" sugiere que también apela a un aspecto estético dentro del segmento económico. Evidencia: Source #0, #5, #9.

**1. Sentimiento general sobre la moto**
El sentimiento general es positivo, centrado en su diseño atractivo, precio económico y características como la iluminación LED completa. Existe una preocupación específica sobre su rendimiento en pendientes debido al motor 125cc. Evidencia: Source #0, #2, #7.

**2. Sensaciones de manejo más mencionadas**
- **Hecho:** El motor incorpora tecnología EBT que reduce las vibraciones y mejora el consumo de combustible. Evidencia: Source #4.
- **Inferencia:** Esto debería traducirse en una experi

In [15]:
# (celda eliminada: la llamada al modelo ahora está en Pase 1 y Pase 2)

Ejecutando DeepSeek en Replicate: deepseek-ai/deepseek-v3.1
Listo en 37.1s

